In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.base import clone
#import category_encoders as ce
import time
import collections
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
import math
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pathlib import Path
import os
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
import statsmodels.api as sm
from sklearn.model_selection import RandomizedSearchCV
#from category_encoders import TargetEncoder


In [2]:
# import data that has already been preprocessed
X_train = pd.read_csv('X_train.csv',index_col=0)
y_train = pd.read_csv('y_train.csv',index_col=0)
X_test = pd.read_csv('X_test.csv',index_col=0)
y_test = pd.read_csv('y_test.csv',index_col=0)
X_train.head()

,Patient_age,Genes_mother_side,Inherited_father,Maternal_gene,Paternal_gene,Blood_cell_count,Status,Respiratory_rate,Heart_rate,Follow_up,...,History_previous_pregnancies,Previous_abortions,Birth_defects,White_blood_cell_count,Blood_test,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5
13647,-1.189643,0.852120,1.487100,0.589244,0.504979,-1.725400,0.961214,-0.802729,-0.669551,-0.928280,...,-0.011244,-0.382844,-0.394093,-2.016210,1.179975,0.901598,1.177452,-2.068385,-1.643148,-1.565422
16490,-1.428746,-0.370583,-0.743214,-0.985130,0.359504,1.292550,-0.331315,-0.624397,-0.607944,1.677702,...,-0.214748,-0.556007,0.388358,-0.578357,-1.513910,1.763845,-0.765864,0.875969,1.495916,-1.228400
4638,-0.472331,-0.352353,-0.743214,-1.775636,0.504979,0.468095,0.961214,-0.832332,-0.669551,0.125815,...,-0.011244,-0.917426,-1.597768,-0.419822,1.179975,0.901598,-1.088203,-2.068385,0.452145,-1.565422
1364,0.723188,0.063834,1.487100,-1.424639,-0.904261,0.417500,-0.467394,-0.138618,-1.459780,-0.294799,...,1.772549,1.818042,1.494336,0.190584,-0.588865,-0.511990,-0.631469,-0.266679,-0.384075,0.932942
15165,0.484084,0.614918,1.487100,-0.416620,1.683296,0.927449,1.415094,0.294454,0.847785,1.138934,...,1.185632,0.161856,1.692602,-0.541987,-1.143906,0.413652,-0.763680,0.562028,0.785125,0.394824


## Train/val split

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.20, random_state=42, shuffle=True,stratify=y_train)
print ('Train observations: %d\nvalidation observations: %d' % (X_train.shape[0], X_val.shape[0]))

Train observations: 12759
validation observations: 3190


## Fine Tuning

## Logistic regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform

In [5]:
# Create a Logistic Regression pipeline with RandomizedSearchCV
lr_pipeline = Pipeline([ 
    ('lr', RandomizedSearchCV(
        LogisticRegression(max_iter=1000, random_state=42),
        {
            'C': [0.001, 0.01, 0.1, 1, 10, 100],  
            'multi_class': ['ovr', 'multinomial', 'auto'],
            'penalty': ['l2'],
            'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
            'max_iter': [100, 500, 1000, 5000]
        },
        n_iter=10,  # Number of parameter settings that are sampled
        cv=5,  # Number of cross-validation folds
        n_jobs=-1,
        verbose=3
    ))  # Logistic Regression model with hyperparameter tuning
])

In [6]:
lr_pipeline.fit(X_train, y_train.values.ravel())
best_lr_model = lr_pipeline.named_steps['lr'].best_estimator_
best_lr_model

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\disle\miniconda3\envs\base_ds_v4\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\disle\miniconda3\envs\base_ds_v4\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\disle\miniconda3\envs\base_ds_v4\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\disle\miniconda3\envs\base_ds_v4\lib\site-packages\sklearn\linear_model\_logistic.py", line 1218, in 

LogisticRegression(C=10, max_iter=500, random_state=42)

In [7]:
# Evaluate best Logistic Regression model
pred_val_lr_tuned = best_lr_model.predict(X_test)
print("Tuned Logistic Regression Classification Report:")
print(classification_report(y_test, pred_val_lr_tuned))

Tuned Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.52      0.98      0.68      2041
           1       0.47      0.13      0.20       414
           2       0.56      0.01      0.02      1533

    accuracy                           0.52      3988
   macro avg       0.52      0.37      0.30      3988
weighted avg       0.53      0.52      0.38      3988



## Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

rand = Pipeline([
    ("rand", RandomForestClassifier(criterion="gini"))
])

In [9]:
# Hyperparameter tuning using RandomizedSearchCV
param_dist = {
    'rand__n_estimators': [10, 50, 100, 200],
    'rand__max_depth': [None, 10, 20, 30],
    'rand__min_samples_split': [2, 5, 10],
    'rand__min_samples_leaf': [1, 2, 4]
}

In [10]:
random_search = RandomizedSearchCV(rand, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1)
random_search.fit(X_train, y_train.values.ravel())
best_rand_model = random_search.best_estimator_
best_rand_model

Pipeline(steps=[('rand',
                 RandomForestClassifier(max_depth=30, min_samples_leaf=2,
                                        min_samples_split=5))])

In [11]:
# Evaluate best RandomForest model
pred_test_rf_tuned = best_rand_model.predict(X_test)
print("Tuned RandomForest Classification Report:")
print(classification_report(y_test, pred_test_rf_tuned))

Tuned RandomForest Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.84      0.68      2041
           1       0.48      0.13      0.20       414
           2       0.52      0.30      0.38      1533

    accuracy                           0.56      3988
   macro avg       0.52      0.42      0.42      3988
weighted avg       0.54      0.56      0.51      3988



## XGBoost

In [12]:
from xgboost import XGBClassifier

In [13]:
# Create an XGBoost pipeline with RandomizedSearchCV
xgb_pipeline = Pipeline([
    ('xgb', RandomizedSearchCV(
        XGBClassifier(),
        {
            'learning_rate': [0.01, 0.1, 0.2, 0.3], 
            'n_estimators': [50, 100, 200, 300],  
            'max_depth': [3, 5, 7, 10],  
            'min_child_weight': [1, 3, 5],  
            'subsample': [0.8, 0.9, 1.0],  
            'colsample_bytree': [0.8, 0.9, 1.0],  
            'gamma': [0, 1, 5],  
            'reg_alpha': [0, 0.1, 0.5, 1], 
            'reg_lambda': [0, 1, 5] 
            
        },
        n_iter=10,
        cv=5,
        n_jobs=-1,
        verbose=3
    ))
])

In [14]:
# Fit the pipeline to the training data
xgb_pipeline.fit(X_train, y_train.values.ravel())

Fitting 5 folds for each of 10 candidates, totalling 50 fits


Pipeline(steps=[('xgb',
                 RandomizedSearchCV(cv=5,
                                    estimator=XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            gamma=None,
                                                            gpu_id=None,
                                                            grow_policy=None,
                                                            importance_type=None,
                                                            interaction_constra...
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            predictor=None,
                                                            random_state=None, ...),
                                    n_jobs=-1,
                                    param_distributions={'colsample_bytree': [0.8,
                                                                              0.9,
                                                                              1.0],
                                                         'gamma': [0, 1, 5],
                                                         'learning_rate': [0.01,
                                                                           0.1,
                                                                           0.2,
                                                                           0.3],
                                                         'max_depth': [3, 5, 7,
                                                                       10],
                                                         'min_child_weight': [1,
                                                                              3,
                                                                              5],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          200,
                                                                          300],
                                                         'reg_alpha': [0, 0.1,
                                                                       0.5, 1],
                                                         'reg_lambda': [0, 1,
                                                                        5],
                                                         'subsample': [0.8, 0.9,
                                                                       1.0]},
                                    verbose=3))])

In [15]:
# Access the best estimator from the RandomizedSearchCV
best_xgb_model = xgb_pipeline.named_steps['xgb'].best_estimator_
best_xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [16]:
pred_test_xgb_tuned = best_xgb_model.predict(X_test)
print("Tuned XGBoost Report:")
print(classification_report(y_test, pred_test_xgb_tuned))

Tuned XGBoost Report:
              precision    recall  f1-score   support

           0       0.57      0.89      0.70      2041
           1       0.51      0.20      0.28       414
           2       0.56      0.24      0.34      1533

    accuracy                           0.57      3988
   macro avg       0.55      0.44      0.44      3988
weighted avg       0.56      0.57      0.51      3988



In [17]:
# Model ensemble with the best models
ensemble_model = VotingClassifier(
    estimators=[
        ('tuned_lr', best_lr_model),
        ('tuned_rf', best_rand_model),
        ('tuned_xgb', best_xgb_model)
    ],
    voting='hard'
)


In [18]:
ensemble_model.fit(X_train, y_train.values.ravel())

VotingClassifier(estimators=[('tuned_lr',
                              LogisticRegression(C=10, max_iter=500,
                                                 random_state=42)),
                             ('tuned_rf',
                              Pipeline(steps=[('rand',
                                               RandomForestClassifier(max_depth=30,
                                                                      min_samples_leaf=2,
                                                                      min_samples_split=5))])),
                             ('tuned_xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8,
                                            e...
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=7,
                                            max_leaves=None, min_child_weight=3,
                                            missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob',
                                            predictor=None, ...))])

In [19]:
# Evaluate ensemble model
pred_test_ensemble = ensemble_model.predict(X_test)
print("Ensemble Model Classification Report:")
print(classification_report(y_test, pred_test_ensemble))

Ensemble Model Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.92      0.70      2041
           1       0.52      0.13      0.21       414
           2       0.59      0.20      0.30      1533

    accuracy                           0.56      3988
   macro avg       0.56      0.42      0.40      3988
weighted avg       0.57      0.56      0.49      3988

